In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

Using TensorFlow backend.


In [3]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.25))
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 5, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [4]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

training_set = datagen.flow_from_directory('train',
                                            target_size = (64, 64),
                                            shuffle = True,
                                            batch_size = 32,
                                            seed= 101,
                                            class_mode = 'categorical',
                                            subset="training")

test_set = datagen.flow_from_directory('train',
                                        target_size = (64, 64),
                                        shuffle = True,
                                        batch_size = 32,
                                        seed = 101,
                                        class_mode = 'categorical',
                                        subset="validation")

Found 6179 images belonging to 5 classes.
Found 1542 images belonging to 5 classes.


In [5]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 6179,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 1542)

Epoch 1/5
  71/6179 [..............................] - ETA: 1:03:45 - loss: 1.3677 - acc: 0.3759

/home/zohebabai/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6029312 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/zohebabai/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1311848 bytes but only got 785. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/zohebabai/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


6178/6179 [============================>.] - ETA: 0s - loss: 0.4042 - acc: 0.8472

/home/zohebabai/anaconda3/lib/python3.6/site-packages/PIL/Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


6179/6179 [==============================] - 970s 157ms/step - loss: 0.4042 - acc: 0.8472 - val_loss: 0.6918 - val_acc: 0.8079
Epoch 2/5
6179/6179 [==============================] - 849s 137ms/step - loss: 0.1757 - acc: 0.9338 - val_loss: 0.7595 - val_acc: 0.8148
Epoch 3/5
6179/6179 [==============================] - 841s 136ms/step - loss: 0.1222 - acc: 0.9558 - val_loss: 0.8322 - val_acc: 0.8249
Epoch 4/5
6179/6179 [==============================] - 839s 136ms/step - loss: 0.0941 - acc: 0.9671 - val_loss: 0.8460 - val_acc: 0.8221
Epoch 5/5
6179/6179 [==============================] - 842s 136ms/step - loss: 0.0796 - acc: 0.9730 - val_loss: 0.9723 - val_acc: 0.8192


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory('Test',
                                                  target_size=(64, 64),
                                                  batch_size=1,
                                                  class_mode=None,
                                                  shuffle=False,
                                                  seed=101)

Found 856 images belonging to 1 classes.


In [7]:
test_generator.reset()
pred=classifier.predict_generator(test_generator,verbose=1)

856/856 [==============================] - 2s 3ms/step


In [9]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)

In [10]:
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [12]:
import pandas as pd
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("submission.csv",index=False)